# Libraries

In [89]:
%matplotlib inline
import cv2
import os

import numpy as np
import mediapipe as mp

# Import mediapipe model for finger and pose detection

In [90]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

# Functions

In [91]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [92]:
def draw_landmarks(image, results):
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [93]:
def draw_styled_landmarks(image, results):
    # # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [116]:
def extract_keypoints(results):
    if results.pose_landmarks:
        pose = np.array([[po.x, po.y, po.z] for po in results.pose_landmarks.landmark]).flatten()
    else:
        pose = np.zeros(33*3)

    if results.left_hand_landmarks:
        left = np.array([[lh.x, lh.y, lh.z] for lh in results.left_hand_landmarks.landmark]).flatten()
    else:
        left = np.zeros(21*3)

    if results.right_hand_landmarks:
        right = np.array([[rh.x, rh.y, rh.z] for rh in results.right_hand_landmarks.landmark]).flatten()
    else:
        right = np.zeros(21*3)
    
    return np.concatenate([pose, left, right])

In [99]:
def extract_coordinates(filepath, savepath):
    cap = cv2.VideoCapture(filepath)
    frame_num = 1
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        # while(frame_num < 31):
        while cap.isOpened():
            ret, frame = cap.read()

            if ret == True:
            
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)
                keypoints = extract_keypoints(results)

                save_dest = os.path.join(savepath, str(frame_num))

                np.save(save_dest, keypoints)

                frame_num += 1
            
                cv2.imshow('frame', image)
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
 
            else:
                break
    
    cap.release()
    cv2.destroyAllWindows()

    return frame_num

In [96]:
def count_files_in_dir(directory_path):
    count = 0
    for path in os.listdir(directory_path):
        if os.path.isfile(os.path.join(directory_path, path)):
            count += 1
    return count

# Folder creation

Create folders for
1. Label folders for new actions
2. Sub-label folders

In [97]:
# set array of actions, may be updated when more data samples are added
actions = np.array(sorted(['door', 'house', 'again', 'open'])) # sorted to follow folder arrangement

# current directory
c_dir = os.getcwd()

In [98]:
c_dir

'd:\\GitHub\\SSLrecognition\\train_data'

# Keypoint extraction
Here we extract the keypoints of the data (videos) by looping through each set of videos in each action folder, then writing the corresponding keypoints. 

This will be used later for model training.

In [130]:
video_count = 0 # required for processing later
for action in actions:
    counter = 0
    for video in os.listdir(os.path.join(c_dir, 'videos', action)):
        counter += 1
        video_count += 1
        filepath = os.path.join(os.path.join(c_dir, 'videos', action, video))

        # NOTE: for each video, save keypoints in new subfolder
        # create subfolder if it does not exist
        subfolder = str(action) + '_' + str(counter)
        if not os.path.isdir(os.path.join(c_dir, 'labels', action, subfolder)):
            os.mkdir(os.path.join(c_dir, 'labels', action, subfolder))
        # set new savepath
        savepath = os.path.join(c_dir, 'labels', action, subfolder)
        print(f'Currently reading video {video_count}: {filepath}')
        extract_coordinates(filepath, savepath)


Currently reading video 1: d:\GitHub\SSLrecognition\train_data\videos\again\fq_again_1.mp4


c:\Users\wei-z\anaconda3\envs\slproj\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Currently reading video 2: d:\GitHub\SSLrecognition\train_data\videos\again\fq_again_2.mp4
Currently reading video 3: d:\GitHub\SSLrecognition\train_data\videos\again\zh_again_1.mp4
Currently reading video 4: d:\GitHub\SSLrecognition\train_data\videos\again\zh_again_2.mp4
Currently reading video 5: d:\GitHub\SSLrecognition\train_data\videos\again\zh_again_3.mp4
Currently reading video 6: d:\GitHub\SSLrecognition\train_data\videos\door\fq_door_1.mp4
Currently reading video 7: d:\GitHub\SSLrecognition\train_data\videos\door\k_door_1.mp4
Currently reading video 8: d:\GitHub\SSLrecognition\train_data\videos\door\k_door_2.mp4
Currently reading video 9: d:\GitHub\SSLrecognition\train_data\videos\door\k_door_3.mp4
Currently reading video 10: d:\GitHub\SSLrecognition\train_data\videos\door\zh_door_1.mp4
Currently reading video 11: d:\GitHub\SSLrecognition\train_data\videos\door\zh_door_2.mp4
Currently reading video 12: d:\GitHub\SSLrecognition\train_data\videos\door\zh_door_3.mp4
Currently rea

# Preprocessing

In [101]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences

In [102]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'again': 0, 'door': 1, 'house': 2, 'open': 3}

In [118]:
sequences, labels = [], []
for action in actions:
    no_actions = len(os.listdir(os.path.join(c_dir, 'labels', action)))
    print('Opening path:', os.path.join(c_dir, 'labels', action))
    print(f'Number of instances: {no_actions}')
    for num in range(1, no_actions + 1):
        window = []
        file = str(action) + "_" + str(num)
        no_frames_per_action = len(os.listdir(os.path.join(c_dir, 'labels', action, file)))
        print(f'Number of frames in {file}: {no_frames_per_action}')
        for frame_num in range(1, no_frames_per_action + 1):
            res = np.load(os.path.join(c_dir, 'labels', action, file,  "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    print('-'*75)

Opening path: d:\GitHub\SSLrecognition\train_data\labels\again
Number of instances: 5
Number of frames in again_1: 96
Number of frames in again_2: 96
Number of frames in again_3: 70
Number of frames in again_4: 57
Number of frames in again_5: 55
---------------------------------------------------------------------------
Opening path: d:\GitHub\SSLrecognition\train_data\labels\door
Number of instances: 7
Number of frames in door_1: 87
Number of frames in door_2: 78
Number of frames in door_3: 78
Number of frames in door_4: 73
Number of frames in door_5: 42
Number of frames in door_6: 50
Number of frames in door_7: 37
---------------------------------------------------------------------------
Opening path: d:\GitHub\SSLrecognition\train_data\labels\house
Number of instances: 4
Number of frames in house_1: 105
Number of frames in house_2: 57
Number of frames in house_3: 59
Number of frames in house_4: 67
---------------------------------------------------------------------------
Opening p

In [119]:
x = np.array(pad_sequences(sequences, dtype = 'float', padding = 'post', value = 0))
y = pad_sequences(to_categorical(labels).astype(int), dtype = 'int', padding = 'post', value = -1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

In [134]:
x_train

array([[[ 0.51165366,  0.30538797, -0.87225628, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.512703  ,  0.30539393, -0.80433106, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.5139817 ,  0.30593839, -0.79160434, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.51908654,  0.28268555, -0.81520557, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.51934218,  0.27510527, -0.79337794, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.51995975,  0.26553988, -0.82854897, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.46093932,  0.43097317, -0.43279457, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.46076268,  0.42749408, -0.43211746, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.46126556,  0.42596292, -0.43918505, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

# Models

## LSTM

In [79]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard, TerminateOnNaN

In [80]:
log_dir = os.path.join(c_dir, 'Logs')
tb_callback = TensorBoard(log_dir = log_dir)

term = TerminateOnNaN()

In [125]:
model = Sequential()
model.add(LSTM(64, return_sequences = True, activation = "relu", input_shape = (117, 225)))
model.add(LSTM(128, return_sequences = True, activation = "relu"))
model.add(LSTM(64, return_sequences = False, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(actions.shape[0], activation = "softmax"))

In [126]:
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])

In [127]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 117, 64)           74240     
                                                                 
 lstm_16 (LSTM)              (None, 117, 128)          98816     
                                                                 
 lstm_17 (LSTM)              (None, 64)                49408     
                                                                 
 dense_15 (Dense)            (None, 64)                4160      
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 4)                 132       
                                                                 
Total params: 228836 (893.89 KB)
Trainable params: 228

In [128]:
model.fit(x_train, y_train, epochs = 2000, callbacks = [tb_callback, term])

Epoch 1/2000


1/1 [==============================] - 2s 2s/step - loss: 1.3718 - categorical_accuracy: 0.4444
Epoch 2/2000
1/1 [==============================] - 0s 83ms/step - loss: 40037560320.0000 - categorical_accuracy: 0.2222
Epoch 3/2000
1/1 [==============================] - 0s 81ms/step - loss: 13305771008.0000 - categorical_accuracy: 0.5000
Epoch 4/2000
1/1 [==============================] - 0s 77ms/step - loss: 45638750208.0000 - categorical_accuracy: 0.4444
Epoch 5/2000
1/1 [==============================] - 0s 79ms/step - loss: 495691759616.0000 - categorical_accuracy: 0.2222
Epoch 6/2000
1/1 [==============================] - 0s 81ms/step - loss: 189735534592.0000 - categorical_accuracy: 0.2222
Epoch 7/2000
1/1 [==============================] - 0s 77ms/step - loss: 65002348544.0000 - categorical_accuracy: 0.1667
Epoch 8/2000
1/1 [==============================] - 0s 79ms/step - loss: 12010512384.0000 - categorical_accuracy: 0.1667
Epoch 9/2000
1/1 [==============================] - 0s 

In [129]:
res = model.predict(x_test)
print(res)
print(y_test)

1/1 [==============================] - 0s 257ms/step
[[2.71361530e-01 4.44482982e-01 1.06366265e-07 2.84155399e-01]
 [2.71361321e-01 4.44483429e-01 1.06366272e-07 2.84155130e-01]]
[[1 0 0 0]
 [0 0 1 0]]
